# 三维高斯喷溅 3D Gaussian Splatting 

- References:
  - [【较真系列】讲人话-3d gaussian splatting全解(原理+代码+公式)](https://space.bilibili.com/644569334/channel/collectiondetail?sid=3173291&spm_id_from=333.788.0.0)
  - [ 3D Gaussian Splatting 代码解读第二期 part1 (forward.cu)](https://www.bilibili.com/video/BV144421U7qb)

## 1. 3D Gaussian：概率论，椭球

- 什么是 Splatting？
  - 一种 *体渲染*（Volumetric Rendering）的方法，把 3D 物体渲染到 2D 平面上
    - Ray Casting 是 *被动* 的
      - 计算出每个 像素点 受到的 发光粒子 的影响
      - NeRF 中用的体渲染算法
    - Splatting 是 *主动* 的
      - 计算出每个 发光粒子 如何影响到 像素点
        - 影响到哪些粒子？靠 *扔雪球* splat 一下出来的 footprint
      - 1990 年的老东西了，快，但效果不好
      - 3DGS 基于 2001 年 EWA Volume Splatting
  - Splat，拟声词，bia 唧一声
    - 把雪球扔到墙上，留下一个引子，这个印子称作 footprint
  - Splatting 的核心：
    - 选择雪球
    - 抛雪球，得到 footprint
    - 对 footprint 进行合成 blending，生成图像
- 为什么选择 3D Gaussian 作为雪球？
  - 为什么需要一个 *核*？
    - 体渲染基于点云
    - 点是没有体积的，需要给点一个核
    - 核可以是 高斯，圆，正方体，…
  - 高斯有一个很好的数学性质：仿射变换后高斯核依旧闭合
    - 仿射变换 $\mathbf{w} = \mathbf{A} \mathbf{x} + \mathbf{b}$
      - 等价于 齐次坐标 下的 geometric transformation
    - 3D Gaussian 降维到二维之后，依然是 2D Gaussian
      - 这里的降维在数学上体现为：沿着某个坐标轴积分

- 3D Gaussian
  - 椭球高斯：$\displaystyle G(\mathbf{x}) = \dfrac{1}{\sqrt{(2 \pi)^k |\mathbf{\Sigma}|}} \exp \left( -\dfrac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\top} \mathbf{\Sigma}^{-1} (\mathbf{x} - \mathbf{\mu}) \right)$
  - $\mathbf{\mu} = (\mu_1, \mu_2, \mu_3)^{\top}$ 是均值
  - $\displaystyle \mathbf{\Sigma} = \begin{pmatrix} {\sigma_1}^2 & \sigma_{xy} & \sigma_{xz} \\ \sigma_{xy} & {\sigma_2}^2 & \sigma_{yz} \\ \sigma_{xz} & \sigma_{yz} & {\sigma_3}^2  \end{pmatrix}$ 是 协方差矩阵，对称，半正定，$|\mathbf{\Sigma}|$ 为其行列式
- 3D Gaussian 为什么是椭球？
  - 一句话：3D Gaussian 的 等概率密度面 iso-surface 是椭球面
    - PDF 的值为常数，那么 exp 的指数为常数，即
    - $-\dfrac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\top} \mathbf{\Sigma}^{-1} (\mathbf{x} - \mathbf{\mu}) = \mathrm{constant}$
    - 展开就是 $ \dfrac{(x - \mu_1)^2}{{\sigma_1}^2} + \dfrac{(y - \mu_2)^2}{{\sigma_2}^2} + \dfrac{(z - \mu_3)^2}{{\sigma_3}^2} - \dfrac{2 \sigma_{xy} (x - \mu_1)(y - \mu_2)}{\sigma_1 \sigma_2} - \dfrac{2 \sigma_{xz} (x - \mu_1)(z - \mu_3)}{\sigma_1 \sigma_3} - \dfrac{2 \sigma_{yz} (y - \mu_2)(z - \mu_3)}{\sigma_2 \sigma_3} = \mathrm{constant}$
  - 各项同性：所有方向有相同的扩散程度，标准圆球，协方差矩阵为 $\sigma^2 \mathbf{I}$；
  - 各项异性：不同方向有不同的扩散程度，椭球。
- 协方差矩阵是如何控制椭球形状的？
  - 高斯分布：
    - $\mathbf{x} \in \mathbb{R}^3 \sim \mathcal{N}(\mathbf{\mu}, \mathbf{\Sigma})$
    - 均值 $\mathbf{\mu} = (\mu_1, \mu_2, \mu_3)^{\top}$ 
    - 协方差矩阵 $\displaystyle \mathbf{\Sigma} = \begin{pmatrix} {\sigma_x}^2 & \sigma_{xy} & \sigma_{xz} \\ \sigma_{xy} & {\sigma_y}^2 & \sigma_{yz} \\ \sigma_{xz} & \sigma_{yz} & {\sigma_z}^2  \end{pmatrix}$
  - 标准高斯分布：
    - $\mathbf{x} \in \mathbb{R}^3 \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
    - 均值 $\mathbf{0} = (0, 0, 0)^{\top}$ 
    - 协方差矩阵 $\displaystyle \mathbf{I} = \begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}$
  - 高斯分布的仿射变换：
    - $\mathbf{x} \in \mathbb{R}^3 \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
    - $\mathbf{w} = \mathbf{A} \mathbf{x} + \mathbf{b}$
    - 那么 $\mathbf{w} \sim \mathcal{N}(\mathbf{A} \mathbf{\mu} + \mathbf{b}, \mathbf{A} \mathbf{\Sigma} \mathbf{A}^{\top})$
  - 即，任何高斯分布都可以视作标准高斯分布通过仿射变换得到的
    - 给定仿射变换 $\mathbf{A}$，$\mathbf{b}$
    - 新的均值为 $\mathbf{\mu} = \mathbf{b}$
    - 新的协方差为 $\mathbf{\Sigma} = \mathbf{A} \mathbf{A}^{\top}$
- 协方差矩阵为什么能用旋转和缩放矩阵表达出来？
  - $\mathbf{A} = \mathbf{R} \mathbf{S}$，则 $\mathbf{\Sigma} = \mathbf{A} \mathbf{A}^{\top} = \mathbf{R} \mathbf{S} \mathbf{S}^{\top} \mathbf{R}^{\top}$
  - 仿射变换角度的解释：
    - 仿射变换包括旋转、缩放和平移
    - 平移对应 $\mathbf{b}$，那么旋转和缩放必然对应 $\mathbf{A}$
  - 几何角度的解释：
    - 单位圆球变成泛化的椭球，一定是通过平移和缩放完成
  - 线性代数角度的解释：
    - 协方差矩阵对称且半正定
    - 这样的矩阵可以被特征值分解为 $\mathbf{\Sigma} = \mathbf{Q} \mathbf{\Lambda} \mathbf{Q}^{\top} = \mathbf{Q} \mathbf{\Lambda}^{\frac{1}{2}} {\mathbf{\Lambda}^{\frac{1}{2}}}^{\top} \mathbf{Q}^{\top}$，其中：
      - $\mathbf{Q}$ 为正交矩阵，对应旋转矩阵；
      - $\mathbf{\Lambda}$ 为对角矩阵，对应缩放矩阵和他自己的转置相乘。
- `computeCov3D` 这个 CUDA kernel 就是在用旋转和缩放矩阵来算协方差矩阵
  - [`__device__ void computeCov3D(const glm::vec3 scale, float mod, const glm::vec4 rot, float * cov3D)`](https://github.com/graphdeco-inria/diff-gaussian-rasterization/blob/9c5c2028f6fbee2be239bc4c9421ff894fe4fbe0/cuda_rasterizer/forward.cu#L114)
  - 没有用旋转矩阵，而是用的四元数（unit quaternion）
    - $\mathbf{rot} = (w, x, y, z) = w + x \mathbf{i} + y \mathbf{j} + z \mathbf{k}$
    - 旋转角度 $ = 2 \arccos(w)$
    - 旋转矩阵（代码里用的左手系）
      $
      \mathbf{R} = 
      \begin{pmatrix}
      1 - 2 (y^2 + z^2) & 2 (xy - rz) & 2 (xz + ry) \\ 
      2 (xy + rz) & 1 - 2 (x^2 + z^2) & 2 (yz - rx) \\ 
      2 (xz - ry) & 2 (yz + rx) & 1 - 2 (x^2 + y^2)
      \end{pmatrix}
      $

## 2. 3D Gaussian：渲染流水线

- 相机模型：连接 3D 世界与 2D 图片
  - 坐标系
    - ![坐标系](https://learnopengl.com/img/getting-started/coordinate_systems.png)
    - 模型坐标系（Model-space Coordinate，Local Coordinate）
    - 世界坐标系（World-space Coordinate）
      - 通过 观测变换 viewing transformation 转换到 相机坐标系：
    - 相机坐标系（Camera-space Coordinate，View-space Coordinate）
      - 通过 投影变换 projection transformation 转换到 归一化相机坐标系：
        - 问题：投影变换不是仿射变换（平行线不再平行）
        - 我们希望 3D Gaussian 一直进行仿射变换
        - 这个通过引入 Jacobian 近似来解决（后面详述）
    - 归一化相机坐标系（Normalized Device Coordinate，NDC，Clip-space Coordinate）
      - 通过 视口变换 viewport transformation 转换到 像素坐标系：
    - 像素坐标系（Screen-space Coordinate）
    - $\mathbf{V}_{\mathrm{clip}} \in [-1, 1]^3 = \mathbf{P} \, \mathbf{V} \,  \mathbf{M} \, \left( \mathbf{V}_{\mathrm{local}} \in \mathbb{R}^3 \right)$
  - 视锥和坐标变换
    - ![视锥](./view_frustum.jpg)
    - Perspective projection is to *stretch* the view frustum into a canonical view volume. 
    - The common practive is to map view-space coordinate $(x, y, z, 1)^\top$ into NDC $(x^{'}, y^{'}, z^{'}, z)^\top = (x_p, y_p, z_p, 1)^\top$.
      - For $x$ and $y$: 
        - The near plane is bounded by $(l, r, b, t)$ where $z = n$;
        - An internal slice inside the frustum where $z = z$ should be bounded by $\left( \dfrac{z}{n} l, \dfrac{z}{n} r, \dfrac{z}{n} b, \dfrac{z}{n} t \right)$;
        - This internal slice should be stretched into $[-1, 1, -1, 1]$;
        - This yields the first two lines in the following equation.
        - Note that the coefficients for $z$ are handled by the $w$ dimension in the actual projection matrix. 
      - For $z$:
        - We need to map $z$ from $[n, f]$ into $[-1, 1]$.
        - This yields the last three lines in the following equation. 
        - Note that because the $w$ dimension is $z$, the mapping is $z_p = \dfrac{Az + B}{z}$
    - 那么：
    $
    \left\{
    \begin{aligned}
    \dfrac{x - \dfrac{z}{n} l }{r - l} & = \dfrac{x_p + 1}{2} \\
    \dfrac{y - \dfrac{z}{n} b }{t - b} & = \dfrac{y_p + 1}{2} \\
    z_p & = \dfrac{A z + B}{z} \\
    z_p(n) & = -1 \\
    z_p(f) & = 1
    \end{aligned}
    \right.
    ;
    $
    - 解得投影变换矩阵如下（参考代码 [getProjectionMatrix](https://github.com/graphdeco-inria/gaussian-splatting/blob/main/utils/graphics_utils.py#L51)）：
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{r - l} & 0 & -\dfrac{r + l}{r - l} & 0 \\
       0 & \dfrac{2n}{t - b} & -\dfrac{t + b}{t - b} & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $
    - 特别地，如果视锥是对称的（$-l = r = \dfrac{w}{2}, -b = t = \dfrac{h}{2}$），则
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{w} & 0 & 0 & 0 \\
       0 & \dfrac{2n}{h} & 0 & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $

- 3D Gaussian 的 观测变换
  - 世界坐标系：
    - 高斯核中心 $t_k = (t_0, t_1, t_2)^{\top}$
    - 高斯核：$r_k^{''}(t) = G_{V_k^{''}}(t - t_k)$
    - $V_k^{''}$ 是协方差矩阵
  - 相机坐标系：
    - 高斯核中心 $u_k = (u_0, u_1, u_2)^{\top}$
    - 高斯核：$r_k^{'}(u) = G_{V_k^{'}}(u - u_k)$
    - 均值 $u_k = \mathrm{affine}(t_k) = W t_k + d$ 
    - 协方差矩阵 $V_k^{'} = W \  V_k^{''} \ W^{\top}$

- 3D Gaussian 的 投影变换
  - 相机坐标系：
    - 高斯核中心 $u_k = (u_0, u_1, u_2)^{\top}$
    - $V_k^{'}$ 是协方差矩阵
  - 投影变换：
    - 高斯核中心 $x_k = (x_0, x_1, x_2)^{\top}$
    - 高斯核：$r_k^(x) = G_{V_k}(x - x_k)$
    - 均值 $x_k = m(u_k)$，$m$ 为投影变换，**非线性**
    - 协方差矩阵 $V_k = J \  V_k^{'} \ J^{\top}$，用 $m$ 的 Jacobian 来近似
      - 论文里的公式：$V_k = J \ W \  V_k^{''} \ W^{\top} \ J^{\top}$，又把 $ V_k^{'}$ 给代换了一次
      - Jacobian 
        - $\mathbf{f}(x, y) = (f_1(x, y), f_2(x, y))^{\top}  = (x + \sin(y), y + \sin(x))^{\top}$
        - $\mathbf{J} = \dfrac{\partial (f_1, f_2)}{\partial (x, y)} = \begin{pmatrix} \dfrac{\partial f_1}{\partial x}& \dfrac{\partial f_1}{\partial y} \\ \dfrac{\partial f_2}{\partial x}& \dfrac{\partial f_2}{\partial y} \end{pmatrix} = \begin{pmatrix} 1 & \cos(y) \\ \cos(x) & 1 \end{pmatrix}$
      - 具体到 投影变换 的 Jacobian：
        - $\mathbf{P} = \begin{pmatrix} focal_x & 0 & 0 & 0 \\ 0 & n & 0 & 0 \\ 0 & 0 & n + f & 0 \\ 0 & 0 & 1 & 0 \end{pmatrix}$
        - $\begin{pmatrix} x_p \\ y_p \\ z_p \end{pmatrix} = \begin{pmatrix} \dfrac{n}{z} x \\ \dfrac{n}{z} y \\ n + f - \dfrac{n}{z} f \end{pmatrix}$
        - $\mathbf{J} = \begin{pmatrix} \dfrac{n}{z} & 0 & -\dfrac{nx}{z^2} \\ 0 & \dfrac{n}{z} & -\dfrac{ny}{z^2} \\ 0 & 0 & -\dfrac{nf}{z^2} \end{pmatrix}$
        - [代码里的公式（横纵焦距不一样）](https://github.com/graphdeco-inria/diff-gaussian-rasterization/blob/9c5c2028f6fbee2be239bc4c9421ff894fe4fbe0/cuda_rasterizer/forward.cu#L89)：$\mathbf{J} = \begin{pmatrix} \dfrac{\mathrm{focal}_x}{z} & 0 & -\dfrac{\mathrm{focal}_x x}{z^2} \\ 0 & \dfrac{\mathrm{focal}_y}{z} & -\dfrac{\mathrm{focal}_y y}{z^2} \\ 0 & 0 & 0 \end{pmatrix}$
  - 此时均值和协方差在同一个坐标系里面吗？
    - 不在！
    - 均值：在 NDC 里，范围 $[-1, 1]^3$
    - 协方差：在 **未缩放** 的正交坐标系里，范围 $[l, r] \times [b, t] \times [f, n]$

- 3D Gaussian 的 视口变换
  - 投影变换后：
    - 高斯核中心 $x_k = (x_0, x_1, x_2)^{\top}$
    - 高斯核：$r_k(x) = G_{V_k}(x - x_k)$
  - 像素坐标系：
    - 高斯核中心 $\mu_k = (\mu_0, \mu_1, \mu_2)^{\top}$
      - 平移 + 缩放
    - 协方差：？
    - 足迹渲染：离散计算
      - $G(x) = \exp \left(-\dfrac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\top} {\mathbf{V}_k}^{-1} (\mathbf{x} - \mathbf{\mu}) \right)$

- 3D Gaussian 变换代码
  - 中心（均值）：
    - `p_hom = get_mvp() @ p_world`：观测，投影变换
    - `p_proj = p_hom / (p_hom[3] + 1e-7)`：归一化（投影变换产生的齐次坐标的 $w$ 维不是 $1$，要修正）
    - `p_screen = viewport_transform(p_proj)`：视口变换
    - 前两项在 [in_frustum](https://github.com/graphdeco-inria/diff-gaussian-rasterization/blob/9c5c2028f6fbee2be239bc4c9421ff894fe4fbe0/cuda_rasterizer/auxiliary.h#L151) 中也有涉及
  - 协方差矩阵：
    - 回顾：
      - $\mathbf{\Sigma}_{\mathrm{clip}} = \mathbf{J} \mathbf{W} \mathbf{\Sigma}_{\mathrm{world}} \left( \mathbf{J} \mathbf{W} \right)^{\top}$
      - $\mathbf{J} = \begin{pmatrix} \dfrac{n}{z} & 0 & -\dfrac{nx}{z^2} \\ 0 & \dfrac{n}{z} & -\dfrac{ny}{z^2} \\ 0 & 0 & -\dfrac{nf}{z^2} \end{pmatrix}$
    - 函数 `computeCov2D(mean, focal_x, focal_y, tan_fovx, tan_fovy, Cov3D, viewMatrix)`：
      - `t = viewMatrix @ mean`：中心点在 **view-space** 的坐标 $(x, y, z)$
        - Jacobi 对应 projection 在 view-space 的局部 Taylor 展开，这就是展开所在的点
        - `projectionMatrix` 是由 `focal` 和 `fov` 算出来的
      - `J = np.array([...])`：按照公式填数即可
      - `W = viewMatrix[:3, :3]`：`J` 是个 $3 \times 3$ 的矩阵，因此 view matrix 需要搞成 $3 \times 3$ 的
      - `cov = (J @ W) @ Cov3D @ (J @ W).T`：按公式乘就完事了
        - 代码里用的是行向量，因此对应矩阵右乘

## 3. 3D Gaussian 球的颜色

- 基函数
  - Fourier 级数收敛到自己的函数可以分解成 sin 和 cos（基函数）的线性组合
  - $\displaystyle f(x) = a_0 + \sum_{n=1}^{\infty} a_n \cos \dfrac{n \pi}{l} x + \sum_{n=1}^{\infty} b_n \sin \dfrac{n \pi}{l} x$
- 球谐函数（Spherical Harmonics）
  - 任何一个球面坐标的函数可以用多个球谐函数来近似
  - $\displaystyle f(t) = \sum_l \sum_{m=-l}^l c_l^m y_l^m(\theta, \phi)$
  - 其中 $l$ 是阶数，$c_l^m$ 是各项系数，值是 RGB 向量；$y_l^m$ 是基函数，值是标量
  - 其实就是把 Fourier 展开给换元了
  - 举例：$3$ 阶球谐函数总共有 $16$ 项求和：$0$ 阶 $1$ 项，$1$ 阶 $3$ 项，$2$ 阶 $5$ 项，$3$ 阶 $7$ 项
    $$
    \begin{aligned}
    f(t) = \phantom{ }
    & c_0^0 y_0^0 + \\
    & c_1^{-1} y_1^{-1} + c_1^{0} y_1^{0} + c_1^{1} y_1^{1} + \\
    & c_2^{-2} y_2^{-2} + c_2^{-1} y_2^{-1} + c_2^{0} y_2^{0} + c_2^{1} y_2^{1} + c_2^{2} y_2^{2} + \\
    & c_3^{-3} y_3^{-3} + c_3^{-2} y_3^{-2} + c_3^{-1} y_3^{-1} + c_3^{0} y_3^{0} + c_3^{1} y_1^{1} + c_3^{2} y_3^{2} + c_3^{3} y_3^{3} \\
    \end{aligned}
    $$
  - 上式中 $0$ 阶的 $y$ 和方向无关，更高阶的 $y$ 通过方向控制 RGB
  $$
  \left\{
  \begin{aligned}
  y_0^0 & = \sqrt{\dfrac{1}{4 \pi}} = 0.28 \\
  y_1^{-1} & = - \sqrt{\dfrac{3}{4 \pi}} \dfrac{y}{r} = -0.49 \dfrac{y}{r} \\
  y_1^{0} & = \sqrt{\dfrac{3}{4 \pi}} \dfrac{z}{r} = 0.49 \dfrac{z}{r} \\
  y_1^{1} & = - \sqrt{\dfrac{3}{4 \pi}} \dfrac{x}{r} = -0.49 \dfrac{x}{r}
  \end{aligned}
  \right.
  $$

- 为什么球谐函数能更好地表达颜色？
  - 直觉上：维度高，参数数量多，储存的信息多
    - RGB，只有 $3$ 个数
    - $3$ 阶球谐函数， $\mathbf{C}$ 矩阵，有 $16 \times 3$ 个数
  - CG：[环境贴图（Environment Map）](https://learnopengl.com/Advanced-OpenGL/Cubemaps)
    - Skybox，rendering volume 的六个面上存贴图，场景内的反射面上就能反射出四周的环境
    - 越高阶的球谐函数，模拟的 skybox 就越逼真
    - 在渲染中是在用球谐函数来编码亮度和颜色

- 足迹合成
  - 直观上：[Alpha Blending](https://learnopengl.com/Advanced-OpenGL/Blending)
  ```python
  sheets = []
  for g in gaussians:
      sheets.append(g.footprint)
  alpha_blending(sheets)
  ```
  - 实际上：g.footprint，依旧对每个像素进行着色
  ```python
  footprint = np.zeros((H, W, 3))
  for i in range(H):
      for j in range(W):
          footprint[i, j] = ...    
  ```

- 像素的颜色
  - 体渲染：对光线上的粒子颜色进行求和【NeRF玩剩下的】
    - 连续版本：
      - $\displaystyle C = \int_0^{+\infty} T(s) \sigma(s) C(s) \mathrm{d}s = \sum_i T_i \alpha_i C_i$
      - $T(s)$：在 s 点之前，光线没有被阻挡的概率，【由两个已知求出来】
        - $T(s) = \exp(-\int_0^s \sigma(t) \mathrm{d}t)$，这是按概率论列微分方程解出来的解析解
      - $\sigma(s)$：在 s 点处，光线碰撞粒子（光线被粒子阻碍）的概率，【已知】
      - $C(s)$：在 s 点处，粒子的颜色，【已知】
    - 离散化：
      - 将光线 $[0, s]$ 划分为 N 个等间距区间 $[T_n, T_{n+1}]$，$n = 0, 1, \cdots, N$
        - 间隔长度为 $\delta_n$
        - 假设区间内的密度 $\sigma_n$ 和颜色 $C_n$ 固定
      - $\displaystyle C(r) = \sum_{n=0}^N C_n e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i} (1 - e^{- \sigma_n \delta_n}) = \sum_n C_n T_n \alpha_n$，其中：
        - $C_n$ 叫颜色，是之前通过球谐函数算出来的 RGB 值
        - $T_n = e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i}$ 叫没有被阻挡的概率（transmittance）
          - $T_n = 0$ 的话直接不用算了，因为被挡住了
        - $\alpha_n = 1 - e^{- \sigma_n \delta_n}$ 叫不透明度（opacity）
          - $\alpha_i$ 由 Gaussian 椭球自身的不透明度，以及当前点距离椭球的距离衰减组成
  - 和 NeRF 一样的公式，凭什么 3DGS 就快呢？
    - Splatting 相对于 Ray Casting，**没有**找粒子的过程
      - Ray Casting 需要对光线进行采样（NeRF 还得采样两次）
      - Splatting 直接就有现成的高斯椭球了
        - 这些椭球需要预先按照深度 $z$ 来排序
    - 粒子数量并没有减少，主要快在 GPU 对每个像素并行处理
      - CUDA 负责 Splatting 部分
      - 分区
        - 整张图分成 $16 \times 16 = 256$ 块，每一块内有若干个像素
        - 对每个高斯也按照上面的区块进行划分
        - 每个 thread block 负责一个区，一个线程负责一个像素

- 3DGS 渲染流程
```python
# 用来扔雪球的墙，初始化成黑色的
out_color = np.zeros((H, W, 3))

# 并行处理每个像素
for i in range(H):
    for j in range(W):
        
        # 每个像素给个初始颜色
        pixf = [i, j]
        C = [0, 0, 0]

        # 由近及远遍历 Gaussian
        for idx in point_list:

            # Initial transmittance：最开始不被遮挡的概率是 1
            T = 1
            
            # 算这个高斯的在这个像素的密度
            xy = points_xy_image[idx]               # Center of 2D Gaussian
            d = [xy[0] - pixf[0], xy[1] - pixf[1]]  # Distance from center of pixel
            con_o = conic_opacity[idx]              # (1.0 / cov_x**2, 1.0 / cov_y**2, 1.0 / cov_xy, alpha)
            
            # 这个像素点离这个高斯太远了，影响不到，直接跳过
            power = -0.5 * (con_o[0] * d[0] * d[0] + con_o[2] * d[1] * d[1]) - con_o[1] * d[0] * d[1]
            if power > 0:
                continue

            # 根据密度 sigma 算不透明度 alpha
            alpha = min(0.99, con_o[3] * np.exp(power))
            if alpha < 1.0 / 255.0:
                continue
            
            # 更新 transmittance，即不被遮挡的概率，如果被挡住了，后面直接不用算了
            test_T = T * (1 - alpha)
            if test_T < 0.0001:
                break

            # 算颜色
            color = features[idx]
            for ch in range(3):
                C[ch] += color[ch] * alpha * T

            T = test_T

        # 最终颜色
        for ch in range(3):
            out_color[j, i, ch] = C[ch] + T * bg_color[ch]

return out_color
```

- 上式中的 `power`：$\displaystyle \dfrac{(x - \mu_1)^2}{{\sigma_1}^2} + \dfrac{(y - \mu_2)^2}{{\sigma_2}^2} + \dfrac{(x - \mu_1)(y - \mu_2)}{\sigma_1 \sigma_2} = \mathrm{constant}$

## 4. 机器学习和参数评估

- 参数：
  - 假设初始点云有 $10000$ 个点
  - 每个点膨胀成 3D Gaussian 椭球，则每个椭球的参数包括：
    - 中心点位置：$(x, y, z)$
    - 协方差矩阵：$\mathbf{R}$，$\mathbf{S}$
    - 球谐函数（Spherical Harmonics）系数：$\mathbf{C}: 16 \times 3$
    - 不透明度（Opacity）：$\alpha$
- 初始化：
  - 依赖 SFM 生成的初始点云
  - 所有椭球都初始化成圆球（isotropic Gaussians）
  - 半径是与 3-近邻的距离的平均值
    - 使用 KNN 找到 3-近邻
- 自适应的高斯控制 Adaptive Control of Gaussians：
  - 太大的（体积占子场景比例太大）：拆分成两个（split）
  - 太小的（体积占子场景比例太小）：复制（clone）
  - 太透明的，或者比视窗还大的：删除（prune）
  - 代码中统称为 [densify](https://github.com/graphdeco-inria/gaussian-splatting/blob/main/scene/gaussian_model.py)
- 算法：体渲染 + SGD
  - Loss：$\mathcal{L} = (1 - \lambda) L_1 + \lambda L_{\mathrm{D-SSIM}}$，$\lambda = 0.2$